In [1]:
import pandas as pd
import numpy as np
import json, ast

def parse_listlike(s):
    if pd.isna(s):               
        return []
    s = str(s).strip()
    if s == '':
        return []
    if s[0] in '[{':
        try:
            return json.loads(s)
        except Exception:
            pass
    try:
        return ast.literal_eval(s)
    except Exception:
        return [s]               


In [2]:
import pandas as pd
df = pd.read_csv('../data/merged_df!!.csv', converters={'section_names': parse_listlike, 'sections': parse_listlike})
df

/tmp/ipykernel_50216/359065799.py:2: DtypeWarning: Columns (69) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/merged_df!!.csv', converters={'section_names': parse_listlike, 'sections': parse_listlike})


,article_id,article_text,abstract_text,labels,section_names,sections,PMID,ids,autobiography,bibliography,...,retrospective_studies,review,scientific_integrity_review,systematic_review,systematic_reviews_as_topic,twin_study,validation_study,veterinary_clinical_trial,veterinary_randomized_controlled_trial,dp
0,PMC3770628,['it occurs in more than 50% of patients and m...,['<S> backgroundanemia in patients with cancer...,NaN,"[Introduction, Patients and methods, Study des...",[[it occurs in more than 50% of patients and m...,24039403,24039403,0,0,...,0,0,0,0,0,0,0,0,0,2013
1,PMC5330001,"['tardive dystonia ( td ) , a rarer side effec...",['<S> tardive dystonia ( td ) is a serious sid...,NaN,"[INTRODUCTION, CASE REPORT, DISCUSSION, Declar...","[[tardive dystonia ( td ) , a rarer side effec...",28250568,28250568,0,0,...,0,0,0,0,0,0,0,0,0,2017
2,PMC4307954,['syncope is caused by transient diffuse cereb...,['<S> we present an unusual case of recurrent ...,NaN,"[Introduction, Case report, Discussion, Confli...",[[syncope is caused by transient diffuse cereb...,25667888,25667888,0,0,...,0,0,0,0,0,0,0,0,0,2014
3,PMC3133512,['agenesis of the inferior vena cava ( ivc ) a...,['<S> background : agenesis of the inferior ve...,NaN,"[Background:, Case:, Conclusion:, Background, ...",[[agenesis of the inferior vena cava ( ivc ) a...,21760745,21760745,0,0,...,0,0,0,0,0,0,0,0,0,2011
4,PMC3056325,['a review of the literature and an extensive ...,['<S> backgrounda review of the literature and...,NaN,"[Background, Case, Conclusion, Background, Cas...",[[a review of the literature and an extensive ...,21403786,21403786,0,0,...,0,0,0,0,0,0,0,0,0,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41849,PMC2526536,['mechanical valve thrombosis ( mvt ) may caus...,"['<S> in the present study , the authors inves...",NaN,"[INTRODUCTION, MATERIALS AND METHODS, RESULTS,...",[[mechanical valve thrombosis ( mvt ) may caus...,18583870,18583870,0,0,...,1,0,0,0,0,0,0,0,0,2008
41850,PMC3853399,['squamous cell carcinoma ( scc ) of the thyro...,['<S> primary squamous cell carcinoma of the t...,NaN,"[INTRODUCTION, CASE REPORT, DISCUSSION]",[[squamous cell carcinoma ( scc ) of the thyro...,24347911,24347911,0,0,...,0,0,0,0,0,0,0,0,0,2013
41851,PMC3324991,['although levodopa ( ld ) still remains the g...,['<S> the aim of this article is to provide a ...,NaN,[Role of dopamine agonists in the treatment of...,[[although levodopa ( ld ) still remains the g...,22500116,22500116,0,0,...,0,1,0,0,0,0,0,0,0,2012
41852,PMC4215460,['fibromyalgia is a chronic pain syndrome of u...,['<S> fibromyalgia is a chronic pain syndrome ...,NaN,"[Introduction, Material and methods, Results, ...",[[fibromyalgia is a chronic pain syndrome of u...,25379438,25379438,0,0,...,0,0,0,0,0,0,0,0,0,2014


In [3]:
import numpy as np
import pandas as pd

# === 1) 取 label 區間（第 8 個到倒數第 2 個；0-based 所以是 7:-1）===
label_cols = df.columns[8:-1].tolist()
Y = df.loc[:, label_cols].astype('uint8').to_numpy(copy=True)   # n x d, 0/1

# === 2) 用 NLL 排除離群（選中間 10%~90% 分位）===
p = Y.mean(axis=0).clip(1e-6, 1-1e-6)
logp, log1mp = np.log(p), np.log(1 - p)
nll = -(Y @ logp + (1 - Y) @ log1mp)          # 每列的「不尋常度」
q_lo, q_hi = np.quantile(nll, [0.10, 0.90])   # 可調，例如 0.2~0.8 更嚴
cand_idx = np.where((nll >= q_lo) & (nll <= q_hi))[0]

# （可選）若資料超大，先隨機下採樣候選集，計算更快
max_candidates = 100_000
if cand_idx.size > max_candidates:
    rng = np.random.default_rng(0)
    cand_idx = rng.choice(cand_idx, size=max_candidates, replace=False)

X = Y[cand_idx].astype(bool)   # 用 bool 讓 XOR 高效
n_cand = X.shape[0]

# === 3) 在候選中做 farthest-point sampling（Hamming 距離）===
k = 25
# 從「最接近 NLL 中位數」的點開始（代表性較高）
start_local = np.argmin(np.abs(nll[cand_idx] - np.median(nll[cand_idx])))
selected_local = [start_local]

min_dist = (X ^ X[start_local]).sum(axis=1)
div_score_at_pick = [np.nan]  

for _ in range(k - 1):
    nxt = np.argmax(min_dist)              
    selected_local.append(nxt)
    div_score_at_pick.append(min_dist[nxt])
    d = (X ^ X[nxt]).sum(axis=1)
    min_dist = np.minimum(min_dist, d)

selected_idx = cand_idx[np.array(selected_local)]

result = (df.iloc[selected_idx, :].copy()
            .assign(nll=nll[selected_idx],
                    min_hamming_at_pick=div_score_at_pick))

result


,article_id,article_text,abstract_text,labels,section_names,sections,PMID,ids,autobiography,bibliography,...,scientific_integrity_review,systematic_review,systematic_reviews_as_topic,twin_study,validation_study,veterinary_clinical_trial,veterinary_randomized_controlled_trial,dp,nll,min_hamming_at_pick
1,PMC5330001,"['tardive dystonia ( td ) , a rarer side effec...",['<S> tardive dystonia ( td ) is a serious sid...,NaN,"[INTRODUCTION, CASE REPORT, DISCUSSION, Declar...","[[tardive dystonia ( td ) , a rarer side effec...",28250568,28250568,0,0,...,0,0,0,0,0,0,0,2017,1.458817,NaN
84,PMC4968912,['this was an observational cohort study of he...,['<S> purposethe purpose of this study was to ...,NaN,"[Methods, Study Population, Standard Automated...",[[this was an observational cohort study of he...,27409505,27409505,0,0,...,0,0,0,0,0,0,0,2016,8.367222,3.0
98,PMC4893752,['early diagnosis of rib fractures can rapidly...,['<S> introduction : the potential benefit of ...,NaN,"[Introduction:, Methods:, Results:, Discussion...",[[early diagnosis of rib fractures can rapidly...,27274514,27274514,0,0,...,0,1,0,0,0,0,0,2016,6.939461,3.0
155,PMC4309867,['this retrospective study included 343 glauco...,['<S> purposeto compare the clinical character...,NaN,"[Materials and Methods, Subjects, Glaucoma pro...",[[this retrospective study included 343 glauco...,25646059,25646059,0,0,...,0,0,0,0,0,0,0,2015,7.787892,3.0
254,PMC5115213,"['the main causes of snhl are advanced age , t...",['<S> backgroundthe purpose of our study was t...,NaN,"[Background, Material and Methods, Subjects, E...","[[the main causes of snhl are advanced age , t...",27846196,27846196,0,0,...,0,0,0,0,0,0,0,2016,8.405738,3.0
5,PMC2778185,['agriculture has been one of the primary econ...,['<S> this was a cross - sectional study that ...,NaN,"[1. Introduction, 2. Methodology, 3. Results a...",[[agriculture has been one of the primary econ...,20041012,20041012,0,0,...,0,0,0,0,0,0,0,2009,5.160773,2.0
8,PMC3155785,['the centers for disease control and preventi...,['<S> \n objective . to examine risk factors f...,NaN,"[1. Introduction, 2. Materials and Methods, 3....",[[the centers for disease control and preventi...,21860798,21860798,0,0,...,0,0,0,0,0,0,0,2012,5.750929,2.0
45,PMC4828917,[''],['<S> mypro is a software pipeline for high - ...,NaN,[Supplementary Material],[[]],25911337,25911337,0,0,...,0,0,0,0,1,0,0,2015,7.006896,2.0
105,PMC3364427,['all study subjects were recruited in a conse...,['<S> purposethe aim of this study was to inve...,NaN,"[Materials and Methods, Optical coherence tomo...",[[all study subjects were recruited in a conse...,22670072,22670072,0,0,...,0,0,0,0,0,0,0,2012,8.366290,2.0
109,PMC3866000,"['between 1903 and 1906 , oswaldo cruz , direc...",['<S> this article addresses the discussion ab...,NaN,[Malaria Campaigns in Brazil and Resistance to...,"[[between 1903 and 1906 , oswaldo cruz , direc...",24331212,24331212,0,0,...,0,0,0,0,0,0,0,2014,6.196268,2.0


In [6]:
for i, row in result.iterrows():
    if row['article_id'] == 'PMC3866000':
        for c in result.columns:
            if row[c] == 1:
                print(c)


historical_article


In [ ]:
keep_cols = ['review','scientific_integrity_review','systematic_review']

label_cols = [c for c in label_cols if c in df.columns]
check_cols = [c for c in label_cols if c not in keep_cols]

# 若沒有需要檢查的欄位，直接不動 df
if len(check_cols) > 0:
    # 轉成數值；非數值→NaN→0
    df[check_cols] = df[check_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

    # 建立遮罩（用 numpy array 避免 index 對不齊）
    mask_other_all_zero = (df[check_cols].to_numpy().sum(axis=1) == 0)

    # 刪掉「其他欄位全為 0」的 row
    df = df.loc[~mask_other_all_zero].copy()

/tmp/ipykernel_1934867/1006373292.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = result.drop(df[mask_other_all_zero].index)


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
keep = df.columns[0:8].tolist()


,article_id,article_text,abstract_text,labels,section_names,sections,PMID,ids
1,PMC5330001,"['tardive dystonia ( td ) , a rarer side effec...",['<S> tardive dystonia ( td ) is a serious sid...,NaN,"[INTRODUCTION, CASE REPORT, DISCUSSION, Declar...","[[tardive dystonia ( td ) , a rarer side effec...",28250568,28250568
84,PMC4968912,['this was an observational cohort study of he...,['<S> purposethe purpose of this study was to ...,NaN,"[Methods, Study Population, Standard Automated...",[[this was an observational cohort study of he...,27409505,27409505
98,PMC4893752,['early diagnosis of rib fractures can rapidly...,['<S> introduction : the potential benefit of ...,NaN,"[Introduction:, Methods:, Results:, Discussion...",[[early diagnosis of rib fractures can rapidly...,27274514,27274514
155,PMC4309867,['this retrospective study included 343 glauco...,['<S> purposeto compare the clinical character...,NaN,"[Materials and Methods, Subjects, Glaucoma pro...",[[this retrospective study included 343 glauco...,25646059,25646059
254,PMC5115213,"['the main causes of snhl are advanced age , t...",['<S> backgroundthe purpose of our study was t...,NaN,"[Background, Material and Methods, Subjects, E...","[[the main causes of snhl are advanced age , t...",27846196,27846196
5,PMC2778185,['agriculture has been one of the primary econ...,['<S> this was a cross - sectional study that ...,NaN,"[1. Introduction, 2. Methodology, 3. Results a...",[[agriculture has been one of the primary econ...,20041012,20041012
8,PMC3155785,['the centers for disease control and preventi...,['<S> \n objective . to examine risk factors f...,NaN,"[1. Introduction, 2. Materials and Methods, 3....",[[the centers for disease control and preventi...,21860798,21860798
45,PMC4828917,[''],['<S> mypro is a software pipeline for high - ...,NaN,[Supplementary Material],[[]],25911337,25911337
105,PMC3364427,['all study subjects were recruited in a conse...,['<S> purposethe aim of this study was to inve...,NaN,"[Materials and Methods, Optical coherence tomo...",[[all study subjects were recruited in a conse...,22670072,22670072
109,PMC3866000,"['between 1903 and 1906 , oswaldo cruz , direc...",['<S> this article addresses the discussion ab...,NaN,[Malaria Campaigns in Brazil and Resistance to...,"[[between 1903 and 1906 , oswaldo cruz , direc...",24331212,24331212


: 

In [ ]:
  # 你要保留的欄名
df_top10 = result.loc[:, keep].copy()
df_top10

In [8]:
out = df_top10.explode(['section_names', 'sections'], ignore_index=True)
out = out.drop(columns=['article_text', 'abstract_text', 'labels', 'PMID', 'ids'])
out

,article_id,section_names,sections
0,PMC5330001,INTRODUCTION,"[tardive dystonia ( td ) , a rarer side effect..."
1,PMC5330001,CASE REPORT,"[we report a case of 20-year - old male , belo..."
2,PMC5330001,DISCUSSION,[earlier case reports reported td developing w...
3,PMC5330001,Declaration of patient consent,[the authors certify that they have obtained a...
4,PMC5330001,Financial support and sponsorship,[]
5,PMC5330001,Conflicts of interest,[]
6,PMC4968912,Methods,[this was an observational cohort study of hea...
7,PMC4968912,Study Population,[this was an observational cohort study of hea...
8,PMC4968912,Standard Automated Perimetry,[standard automated perimetry visual field tes...
9,PMC4968912,Optical Coherence Tomography Angiography Image...,[the angiovue provides a noninvasive oct - bas...


In [9]:
def keep_head_tail(x, k=2):
    if not isinstance(x, list):
        return x
    n = len(x)
    if n <= 4:
        return x
    return x[:k] + x[-k:]

out['sections'] = out['sections'].apply(keep_head_tail)   # k=2 預設
out

,article_id,section_names,sections
0,PMC5330001,INTRODUCTION,"[tardive dystonia ( td ) , a rarer side effect..."
1,PMC5330001,CASE REPORT,"[we report a case of 20-year - old male , belo..."
2,PMC5330001,DISCUSSION,[earlier case reports reported td developing w...
3,PMC5330001,Declaration of patient consent,[the authors certify that they have obtained a...
4,PMC5330001,Financial support and sponsorship,[]
5,PMC5330001,Conflicts of interest,[]
6,PMC4968912,Methods,[this was an observational cohort study of hea...
7,PMC4968912,Study Population,[this was an observational cohort study of hea...
8,PMC4968912,Standard Automated Perimetry,[standard automated perimetry visual field tes...
9,PMC4968912,Optical Coherence Tomography Angiography Image...,[the angiovue provides a noninvasive oct - bas...


In [10]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm

with open('../../section_normalization/structured_abstract_sections.pkl', 'rb') as f:
    normalized_sections = pickle.load(f)


In [15]:
import re

def map_normalized_sections(row, norm_sections):
    mapping = {'background': 'introduction', 'objective': 'introduction', 'methods': 'method', 'results': 'result', 'conclusions': 'discussion'}
    for norm_sec in norm_sections:
        compare = row['section_names'].lower()
        compare = re.sub(r'^\s*\d+(?:\.\d+)*[)\.\u3001\uFF0E]?\s*', '', compare).strip()
        if compare in norm_sections[norm_sec]:
            return mapping[norm_sec]
    return np.nan

In [16]:

# Get normalized data for each section title
out['sec-norm'] = out.apply(lambda row: map_normalized_sections(row, normalized_sections), axis=1)
# data.to_csv('normalized_section_data.csv', index=False)
out

,article_id,section_names,sections,sec-norm
0,PMC5330001,INTRODUCTION,"[tardive dystonia ( td ) , a rarer side effect...",introduction
1,PMC5330001,CASE REPORT,"[we report a case of 20-year - old male , belo...",method
2,PMC5330001,DISCUSSION,[earlier case reports reported td developing w...,discussion
3,PMC5330001,Declaration of patient consent,[the authors certify that they have obtained a...,NaN
4,PMC5330001,Financial support and sponsorship,[],NaN
5,PMC5330001,Conflicts of interest,[],NaN
6,PMC4968912,Methods,[this was an observational cohort study of hea...,method
7,PMC4968912,Study Population,[this was an observational cohort study of hea...,method
8,PMC4968912,Standard Automated Perimetry,[standard automated perimetry visual field tes...,NaN
9,PMC4968912,Optical Coherence Tomography Angiography Image...,[the angiovue provides a noninvasive oct - bas...,NaN


In [17]:
out['sec-norm'].notna().sum()/len(out)

np.float64(0.5932203389830508)

In [18]:
out = out.explode(['sections'], ignore_index=True)
out

,article_id,section_names,sections,sec-norm
0,PMC5330001,INTRODUCTION,"tardive dystonia ( td ) , a rarer side effect ...",introduction
1,PMC5330001,INTRODUCTION,td has been shown to develop in about 3% of pa...,introduction
2,PMC5330001,INTRODUCTION,is thought to have preferential action at meso...,introduction
3,PMC5330001,INTRODUCTION,"furthermore , a retrospective analysis of cont...",introduction
4,PMC5330001,CASE REPORT,"we report a case of 20-year - old male , belon...",method
...,...,...,...,...
203,PMC3866000,Ernst Rodenwalt and Quinine Resistance in the ...,"even so , schilling acknowledged that the way ...",NaN
204,PMC3866000,Concluding Remarks,"today , the resistance of the plasmodium to an...",discussion
205,PMC3866000,Concluding Remarks,the theory of the development of quinine - res...,discussion
206,PMC3866000,Concluding Remarks,"brazilian doctors like arthur neiva , miguel c...",discussion


In [20]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]


In [21]:
out.to_excel('for_train_annotation_trial.xlsx')